In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ==========================
# 1. Series de velocidad
# ==========================
# Waypoint map-matched: velocidad correcta por segmento
wp_speed = df_seg["speed_mph"]

# Trajs: velocidad ya en mph, previamente filtrada (0 < v < 120)
tp_speed = df_tp["CrossingSpeedMph"]

print("Waypoint map-matched count:", len(wp_speed))
print("Trajs count:", len(tp_speed))

# ==========================
# 2. Función para CDF
# ==========================
def make_cdf(series):
    x = np.sort(series.values)
    y = np.arange(1, len(x) + 1) / len(x)
    return x, y

# ==========================
# 3. Crear figura de 4 paneles
# ==========================
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# ---------- Panel 1: Histogramas (densidad) ----------
ax = axes[0, 0]
ax.hist(wp_speed, bins=80, density=True, alpha=0.6, label="waypoint_mapmatched")
ax.hist(tp_speed, bins=80, density=True, alpha=0.6, label="trajs")
ax.set_title("Speed Distribution Comparison", fontsize=14)
ax.set_xlabel("Speed (mph)")
ax.set_ylabel("Density")
ax.legend()

# ---------- Panel 2: Boxplots ----------
ax = axes[0, 1]
ax.boxplot(
    [wp_speed, tp_speed],
    labels=["waypoint_mapmatched", "trajs"],
    patch_artist=True,
    boxprops=dict(facecolor="lightblue", alpha=0.7),
    medianprops=dict(color="darkred")
)
ax.set_title("Speed Distribution Box Plot", fontsize=14)
ax.set_ylabel("Speed (mph)")

# ---------- Panel 3: CDF ----------
ax = axes[1, 0]
wp_x, wp_y = make_cdf(wp_speed)
tp_x, tp_y = make_cdf(tp_speed)

ax.plot(wp_x, wp_y, label="waypoint_mapmatched", linewidth=2)
ax.plot(tp_x, tp_y, label="trajs", linewidth=2)
ax.set_title("Cumulative Distribution Function", fontsize=14)
ax.set_xlabel("Speed (mph)")
ax.set_ylabel("Cumulative Probability")
ax.legend()

# ---------- Panel 4: Summary stats ----------
ax = axes[1, 1]
metrics = ["mean", "median", "std"]
wp_vals = [wp_speed.mean(), wp_speed.median(), wp_speed.std()]
tp_vals = [tp_speed.mean(), tp_speed.median(), tp_speed.std()]

x_idx = np.arange(len(metrics))
width = 0.35

ax.bar(x_idx - width/2, wp_vals, width, label="waypoint_mapmatched")
ax.bar(x_idx + width/2, tp_vals, width, label="trajs")
ax.set_xticks(x_idx)
ax.set_xticklabels(metrics)
ax.set_title("Statistical Summary Comparison", fontsize=14)
ax.set_ylabel("Speed (mph)")
ax.legend()

# Etiquetas numéricas arriba de cada barra
for i, v in enumerate(wp_vals):
    ax.text(i - width/2, v + 0.5, f"{v:.1f}", ha="center", va="bottom", fontsize=8)
for i, v in enumerate(tp_vals):
    ax.text(i + width/2, v + 0.5, f"{v:.1f}", ha="center", va="bottom", fontsize=8)

plt.tight_layout()

# (opcional) guardar
import os
out_dir = os.path.join(os.getcwd(), "figures_from_tool_db_only")
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "speed_distribution_4panel_correct.png")
plt.savefig(out_path, dpi=300)
plt.show()

print("Figure saved to:", out_path)
